# Module 3: Practical Prompt Engineering with GitHub Copilot

#shopping_cart.py
#### Task 1: EDITOR MODE INLINE PROMPT
Prompt: 

Add comment above add_item function: "# Generate test cases for adding items with valid and invalid quantities"

What happens: Copilot suggests test code inline


#### Answer:-

In [ ]:
# ...existing code...
# Generate test cases for adding items with valid and invalid quantities
def add_item(cart, item_id, quantity):
    # ...existing code...
    pass
# ...existing code...

#### Task 2: CHAT MODE ANALYSIS
Prompt:
"Analyze shopping_cart.py and list all testable scenarios"

What happens: Copilot provides detailed test scenario list in chat


#### Answer:-

Below are concise, actionable testable scenarios for shopping_cart.py grouped by feature and type.

Functional

add_item: add new item (valid id, positive qty) — cart updated, item present with correct qty and price total.
add_item: add same item twice — quantities aggregate or replace (specify expected behavior).
remove_item: remove existing item — item removed and totals updated.
update_quantity: change quantity to higher/lower positive integer — totals and item qty updated.
calculate_total: correct sum of item prices * quantities, including decimals.
apply_coupon: valid coupon reduces total by percent/amount.
remove_coupon: coupon removal restores original total.
get_item: fetch item details by id returns correct data.
clear_cart: empties cart and resets totals.
Boundary

add_item: quantity = 0 (expect error or no-op).
add_item: quantity = 1 (minimum valid).
update_quantity: set to 0 (remove vs error — expected behavior).
large quantity (e.g., 1e6) — behavior and performance.
very large cart (many distinct items) — memory/throughput.
Negative / Validation

add_item: negative quantity -> reject (ValueError).
add_item: non-integer quantity (float/string/None) -> reject or coerce per spec.
add_item/remove/update: invalid item_id -> error or no-op.
apply_coupon: expired/invalid coupon -> rejected, no discount applied.
calculate_total: malformed price data (None or string) -> error handling.
operations on empty cart (remove, update, calculate_total) -> defined responses.
Inventory / Business Rules

add_item when stock insufficient -> reject/adjust qty.
price change after adding item -> verify whether cart preserves original price or reflects new price.
coupon minimum-spend not met -> coupon not applied.
coupon stacking rules (one vs multiple) -> correct behavior.
Rounding / Precision

Totals with fractional cents -> rounding rules (banker’s rounding vs round half up).
Tax calculation integration with totals (if present) -> correct order and rounding.
Concurrency / Persistence / Integration

Concurrent add/update from multiple threads/processes -> race conditions.
Save/load cart (session persistence) -> cart state persists across save/load.
Checkout integration with payment system -> totals passed correctly.
Security / Input Safety

Injection strings in item_id/name fields (SQL/HTML) -> inputs sanitized/escaped.
Sensitive data not logged (if cart stores payment info) -> no secrets in logs.

#### Task 3: CONVERSATIONAL REFINEMENT
Prompt: "Generate test cases for calculate_total function" then ask "Add edge cases for empty cart and single item"

What happens: First gives basic tests, second refines with edge cases

#### Answer:-

Here are pytest cases (adjust cart shape to your implementation if needed).


import pytest
from module_3.shopping_cart import calculate_total

def test_calculate_total_empty_cart():
    # empty cart -> zero total
    cart = []  # adjust if your cart is a dict/object
    assert calculate_total(cart) == pytest.approx(0.0)

def test_calculate_total_single_item_quantity_one():
    # single item, qty=1
    cart = [{"id": "sku1", "price": 19.99, "quantity": 1}]
    assert calculate_total(cart) == pytest.approx(19.99)

def test_calculate_total_single_item_multiple_quantity():
    # single item, qty>1
    cart = [{"id": "sku1", "price": 12.50, "quantity": 3}]
    assert calculate_total(cart) == pytest.approx(37.50)

def test_calculate_total_multiple_items_and_rounding():
    # multiple items with fractional prices
    cart = [
        {"id": "a", "price": 0.99, "quantity": 3},
        {"id": "b", "price": 1.25, "quantity": 2},
    ]
    assert calculate_total(cart) == pytest.approx(0.99*3 + 1.25*2)

def test_calculate_total_negative_price_raises():
    # invalid negative price should be rejected
    cart = [{"id": "bad", "price": -5.00, "quantity": 1}]
    with pytest.raises(ValueError):
        calculate_total(cart)

def test_calculate_total_invalid_price_type_raises():
    # malformed price types must raise
    cart = [{"id": "x", "price": None, "quantity": 1}]
    with pytest.raises((TypeError, ValueError)):
        calculate_total(cart)

#### Task 4: ACCEPTANCE CRITERIA TO TEST CONDITIONS
Prompt: "Convert the coupon functionality into detailed test conditions with test steps and expected results"

What happens: Creates structured test conditions from code logic

#### Answer:-

Below are concise, structured test conditions for coupon functionality. Each entry: Test Name | Preconditions | Test Steps | Expected Result.

Apply_Valid_Percent_Coupon | Cart with items totaling $100; coupon = PERC10 (10% off) | 1) Add items to cart totaling $100. 2) Apply coupon code PERC10. | Total reduced by 10% → $90. Discount line shows $10. Coupon marked active on cart.

Apply_Valid_Fixed_Coupon | Cart total $80; coupon = FIXED20 ($20 off) | 1) Build cart = $80. 2) Apply FIXED20. | Total = $60. Discount line shows $20. Total never negative.

Expired_Coupon_Rejected | Cart any; coupon = EXPIRED (past expiry) | 1) Attempt to apply EXPIRED. | Coupon rejected with user-facing message "Coupon expired". No discount applied.

Invalid_Coupon_Code | Cart any; coupon = ABC123 (nonexistent) | 1) Attempt to apply ABC123. | Coupon rejected with "Invalid coupon code". No change to totals.

Minimum_Spend_Not_Met | Cart total $40; coupon MIN50 ($10 off, min spend $50) | 1) Build cart = $40. 2) Apply MIN50. | Coupon rejected with "Minimum spend $50 required". No discount applied.

Coupon_Maximum_Discount_Capped | Cart total $1000; coupon BIGPERC (50% off, cap $200) | 1) Build cart = $1000. 2) Apply BIGPERC. | Discount = min(50% of $1000 = $500, cap $200) => $200. Total = $800. Cap indicated in coupon details.

Coupon_Item_Level_Applicability | Cart contains SKU_A $30, SKU_B $70; coupon SKU_A_20 (20% off SKU_A only) | 1) Add both items. 2) Apply SKU_A_20. | Discount = 20% * $30 = $6. Total = $94. Only SKU_A shows discounted price/line.

Coupon_Stacking_Rules | Cart $200; coupons: PERC10 (10%) + FIXED20 ($20) and stacking disallowed | 1) Apply PERC10. 2) Attempt to apply FIXED20. | If stacking disallowed: second coupon rejected with message "Cannot combine coupons". Total remains after first coupon only. If allowed, verify correct order/priority and final total.

Remove_Coupon_Restores_Total | Cart $150; coupon = FIXED50 | 1) Apply FIXED50 → Total = $100. 2) Remove coupon. | Total returns to $150. Coupon state cleared; discount removed.

Rounding_Precision_With_Percent | Cart items produce fractional discount (e.g., total $33.33); coupon = PERC15 | 1) Cart total = $33.33. 2) Apply 15% coupon. | Discount = 4.9995 -> rounding rule applied (document expected: round half up to $5.00). Total = $28.33. Assert rounding follows spec.

Apply_Coupon_On_Empty_Cart | Empty cart; coupon any valid | 1) Attempt to apply VALID coupon on empty cart. | Either reject with "Cart empty" or accept but discount = $0 and coupon queued — behavior must match spec; assert chosen behavior.

Concurrency_Apply_Same_Coupon | High-concurrency: multiple sessions attempt same single-use coupon | 1) Create coupon single-use. 2) Simultaneously apply from multiple sessions. | Exactly one session succeeds; others receive "Coupon already used". No oversubscription.

Price_Change_After_Apply | Cart $100, apply PERC10, later item price increases to $120 | 1) Add items to $100. 2) Apply PERC10. 3) Update item price to $120. | Define expected: coupon recalculates (new total $108) OR coupon locks original discount amount — assert implemented rule. Test verifies chosen behavior.

Notes: for each test assert:

user-facing messages and error codes,
discount line items and total arithmetic,
coupon state (applied/removed/expired),
no negative totals,
audit/log entry created (if required),
adherence to rounding rules and caps.

#### Task 5: TEST STRATEGY GENERATION
Prompt: "Create test strategy outline for shopping cart covering functional, boundary, and negative testing"

What happens: Generates test strategy document

#### Answer:-

Scope & Objectives

Verify correctness of cart operations (add/remove/update/total/coupons) and business rules.
Ensure robustness against invalid inputs and edge cases.
Provide automation-ready test cases for CI.
Test Types (high level)
Functional: correctness of expected behaviors.
Boundary: edge values and limits.
Negative: invalid inputs, error handling, and resilience.
Non-functional (brief): concurrency, performance for large carts, rounding/precision.

Test Areas (features)

Item operations: add_item, remove_item, update_quantity, get_item, clear_cart
Pricing: calculate_total, rounding, price precision
Coupons: apply, remove, validation, stacking, caps, min-spend
Inventory/business rules: stock checks, price-change semantics
Persistence & integration: save/load, checkout integration
Security & safety: input sanitization, no sensitive logging
Example Test Conditions (grouped)
Functional (High priority)

Add valid item -> item present, correct quantity and subtotal.
Add same item twice -> behavior per spec (aggregate or replace) and totals correct.
Remove existing item -> removed and totals updated.
Apply valid percent coupon -> total reduced by correct percent, discount line present.
Calculate total for mixed items -> total == sum(price * qty) with proper rounding.
    
Non-functional checks (minimal)

Concurrency: simulate concurrent add/update to detect race conditions.
Performance: measure latency for add_item and calculate_total on large cart (N items).
Precision: assert rounding policy (document and enforce).
Acceptance / Exit Criteria
All high-priority functional tests pass.
No critical regressions in negative tests.
Rounding and coupon arithmetic validated against spec.
Performance: calculate_total on 10k items completes within defined SLA (set by team).
No sensitive data in logs during tests.
Prioritization & Next Steps
Priority P0: add/remove/update, calculate_total, coupon apply/remove basic behavior.
Priority P1: boundary cases (qty zero/large), negative inputs.
Priority P2: concurrency, persistence, integration, advanced coupon rules.
Next: convert high-priority items into automated pytest functions and add CI pipeline steps.